In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn.functional as F
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
import os
import time
from torchvision import models, transforms
from torchvision.models import ResNet34_Weights

In [12]:
# ======================
# 安全状态字典加载函数
# ======================
def load_state_dict_safely(model, state_dict):
    """智能处理 DataParallel 和普通模型的键名问题"""
    model_state = model.state_dict()
    
    # 标准化键名：移除所有可能的前缀
    clean_state_dict = {}
    for k, v in state_dict.items():
        # 移除 module. 和 _orig_mod. 前缀
        clean_k = k.replace("module.", "").replace("_orig_mod.", "")
        clean_state_dict[clean_k] = v
    
    # 创建匹配的 state_dict
    matched_state_dict = {}
    for k in model_state.keys():
        clean_k = k.replace("module.", "").replace("_orig_mod.", "")
        if clean_k in clean_state_dict:
            matched_state_dict[k] = clean_state_dict[clean_k]
        else:
            print(f"⚠️ 警告: 键 '{k}' 在state_dict中不存在，使用随机初始化")
    
    model.load_state_dict(matched_state_dict, strict=False)
    return model

In [14]:
# ======================
# 数据集定义
# ======================
class FERDataset(Dataset):
    def __init__(self, image_dir, mode='train', transform=None):
        self.image_dir = Path(image_dir)
        self.mode = mode
        self.transform = transform
        self.samples = []
        self.class_to_idx = {
            "Angry": 0, "Fear": 1, "Happy": 2, 
            "Sad": 3, "Surprise": 4, "Neutral": 5
        }
        self._load_data()

    def _load_data(self):
        if self.mode == "train":
            for emotion_name, label in self.class_to_idx.items():
                emotion_dir = self.image_dir / emotion_name
                if emotion_dir.exists():
                    for img_file in emotion_dir.glob("*"):
                        if img_file.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                            self.samples.append((str(img_file), label))
        else:
            for img_file in self.image_dir.glob("*"):
                if img_file.suffix.lower() in ['.jpg', '.jpeg', '.png']:
                    self.samples.append((str(img_file), -1))

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.mode == 'test':
            return image, img_path
        else:
            return image, label

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, targets):
        ce = F.cross_entropy(
            inputs, targets,
            reduction='none',
            weight=self.alpha
        )
        pt = torch.exp(-ce)
        loss = ((1 - pt) ** self.gamma) * ce
        return loss.mean()


In [ ]:
def mixup_data(x, y, alpha=0.4):
    lam = np.random.beta(alpha, alpha)
    index = torch.randperm(x.size(0)).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index]
    return mixed_x, y, y[index], lam


In [ ]:
@torch.no_grad()
def tta_predict(model, images):
    preds = []
    preds.append(model(images))
    preds.append(model(torch.flip(images, dims=[3])))        # 水平翻转
    preds.append(model(torch.rot90(images, 1, [2, 3])))      # 旋转
    return torch.mean(torch.stack(preds), dim=0)


In [15]:
# ======================
# 模型定义
# ======================
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=8):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y

class AdvancedCNN(nn.Module):
    def __init__(self, out_channels=6, dropout_rate=0.4, pretrained=True):
        super().__init__()
        weights = ResNet34_Weights.IMAGENET1K_V1 if pretrained else None
        self.backbone = models.resnet34(weights=weights)
        self.se = SEBlock(512, reduction=8)
        
        # 替换分类头
        self.backbone.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(256, out_channels)
        )
        
    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)
        x = self.se(x)
        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.backbone.fc(x)
        return x

In [16]:
# ======================
# 数据增强
# ======================
def create_transforms():
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return train_transform, val_transform

In [17]:
# ======================
# 验证函数（修复 AMP 警告）
# ======================
@torch.no_grad()
def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0.0, 0, 0
    device_type = 'cuda' if device.type == 'cuda' else 'cpu'
    
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        with torch.amp.autocast(device_type=device_type):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        total_loss += loss.item()
        _, pred = outputs.max(1)
        total += labels.size(0)
        correct += pred.eq(labels).sum().item()
        
    return total_loss / len(val_loader), 100. * correct / total

In [19]:
# # ======================
# # 训练函数（修复所有警告 + 安全加载）
# # ======================
# def train_model(model, train_loader, val_loader, optimizer, criterion, 
#                 epochs=25, device=None):
#     train_losses, val_losses, train_accs, val_accs = [], [], [], []
    
#     # ✅ 修复 FutureWarning: 使用新 API
#     device_type = 'cuda' if device.type == 'cuda' else 'cpu'
#     try:
#         scaler = torch.amp.GradScaler(device_type)
#     except (AttributeError, TypeError):
#         # 兼容旧版本
#         scaler = torch.cuda.amp.GradScaler() if device.type == 'cuda' else None
    
#     scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-7)
#     best_val_acc = 0.0
#     best_model_path = "best_model.pth"

#     for epoch in range(epochs):
#         model.train()
#         total_loss, correct, total = 0.0, 0, 0
#         start_time = time.time()
        
#         for images, labels in train_loader:
#             images, labels = images.to(device), labels.to(device)
#             optimizer.zero_grad()
            
#             # ✅ 修复 FutureWarning: 使用新 API
#             with torch.amp.autocast(device_type=device_type):
#                 outputs = model(images)
#                 loss = criterion(outputs, labels)
            
#             if device.type == 'cuda':
#                 scaler.scale(loss).backward()
#                 scaler.step(optimizer)
#                 scaler.update()
#             else:
#                 loss.backward()
#                 optimizer.step()
            
#             total_loss += loss.item()
#             _, pred = outputs.max(1)
#             total += labels.size(0)
#             correct += pred.eq(labels).sum().item()
        
#         epoch_time = time.time() - start_time
#         scheduler.step()
#         train_loss = total_loss / len(train_loader)
#         train_acc = 100. * correct / total
#         train_losses.append(train_loss)
#         train_accs.append(train_acc)

#         # Validation
#         val_loss, val_acc = validate(model, val_loader, criterion, device)
#         val_losses.append(val_loss)
#         val_accs.append(val_acc)

#         # 打印详细指标
#         current_lr = scheduler.get_last_lr()[0]
#         print(f"Epoch {epoch+1:02d}/{epochs} | Time: {epoch_time:.1f}s | LR: {current_lr:.2e} | "
#               f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.2f}% | "
#               f"Val Loss: {val_loss:.4f} | Acc: {val_acc:.2f}%")

#         # 仅当验证准确率提升时保存模型
#         if val_acc > best_val_acc:
#             best_val_acc = val_acc
#             if Path(best_model_path).exists():
#                 Path(best_model_path).unlink()
            
#             # 保存原始state_dict
#             state_dict = model.state_dict()
#             # ✅ 仅保存 tensor，避免 numpy 标量
#             torch.save({"state_dict": state_dict}, best_model_path)
#             print(f"  ✅ New best model saved @ {val_acc:.2f}%")

#     # 绘图
#     plot_training_history(train_losses, val_losses, train_accs, val_accs)
    
#     # 核心修复：安全加载最优模型
#     print("\n🔧 安全加载最优模型...")
#     checkpoint = torch.load(best_model_path, map_location=device, weights_only=True)
#     model = load_state_dict_safely(model, checkpoint["state_dict"])
#     return best_model_path, model

In [20]:
# # ======================
# # 测试 & 提交
# # ======================
# def test(model, test_loader, device, output_csv='submission.csv'):
#     model.eval()
#     paths, labels = [], []
#     with torch.no_grad():
#         for images, path_list in test_loader:
#             images = images.to(device)
#             outputs = model(images)
#             _, pred = outputs.max(1)
#             labels.extend(pred.cpu().numpy().tolist())
#             if isinstance(path_list, (list, tuple)):
#                 file_names = [Path(p).name for p in path_list]
#             else:
#                 file_names = [Path(path_list).name]
#             paths.extend(file_names)
    
#     df = pd.DataFrame({"ID": paths, "Emotion": labels})
#     df.to_csv(output_csv, index=False)
#     print(f"✅ Submission saved to '{output_csv}' (rows: {len(df)})")

In [18]:
# ======================
# 可视化
# ======================
def plot_training_history(train_losses, val_losses, train_accs, val_accs):
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    epochs = range(1, len(train_losses) + 1)
    
    # Loss
    ax1.plot(epochs, train_losses, 'b-o', label='Train Loss', linewidth=2, markersize=4)
    ax1.plot(epochs, val_losses, 'r-s', label='Val Loss', linewidth=2, markersize=4)
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.set_title('Loss Curve', fontsize=14)
    ax1.legend(loc='best')
    ax1.grid(alpha=0.3)
    
    # Accuracy
    ax2.plot(epochs, train_accs, 'b-o', label='Train Accuracy', linewidth=2, markersize=4)
    ax2.plot(epochs, val_accs, 'r-s', label='Val Accuracy', linewidth=2, markersize=4)
    
    # 标记最佳点
    best_epoch = val_accs.index(max(val_accs)) + 1
    best_val = max(val_accs)
    ax2.scatter(best_epoch, best_val, color='gold', edgecolor='black', s=150, zorder=5,
                label=f'Best: {best_val:.2f}% @ Epoch {best_epoch}')
    
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Accuracy (%)', fontsize=12)
    ax2.set_title('Accuracy Curve', fontsize=14)
    ax2.legend(loc='best')
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig("training_history.png", dpi=300, bbox_inches='tight')
    plt.close()
    print("📈 Training history saved as 'training_history.png'")

In [ ]:
def train_model(model, train_loader, val_loader, optimizer, criterion,
                epochs=25, device=None):

    device_type = 'cuda' if device.type == 'cuda' else 'cpu'
    scaler = torch.amp.GradScaler(device_type)

    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=epochs, eta_min=1e-7
    )

    best_val_acc = 0.0
    best_model_path = "best_model.pth"

    for epoch in range(epochs):
        model.train()
        total_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            # ===== MixUp =====
            images, y_a, y_b, lam = mixup_data(images, labels)

            with torch.amp.autocast(device_type=device_type):
                outputs = model(images)
                loss = lam * criterion(outputs, y_a) + \
                       (1 - lam) * criterion(outputs, y_b)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()

        scheduler.step()

        # ===== Validation =====
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        print(f"Epoch [{epoch+1}/{epochs}] "
              f"Train Loss: {total_loss/len(train_loader):.4f} | "
              f"Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({"state_dict": model.state_dict()}, best_model_path)
            print(f"  ✅ Best model saved: {val_acc:.2f}%")

    checkpoint = torch.load(best_model_path, map_location=device)
    model.load_state_dict(checkpoint["state_dict"], strict=False)
    return model


In [ ]:
def test(model, test_loader, device, output_csv='submission.csv'):
    model.eval()
    paths, labels = [], []

    with torch.no_grad():
        for images, path_list in test_loader:
            images = images.to(device)

            outputs = tta_predict(model, images)
            _, pred = outputs.max(1)

            labels.extend(pred.cpu().numpy().tolist())
            paths.extend([Path(p).name for p in path_list])

    df = pd.DataFrame({"ID": paths, "Emotion": labels})
    df.to_csv(output_csv, index=False)
    print(f"✅ Submission saved to {output_csv}")


In [21]:
# # ======================
# # 主程序（T4*2 专属，禁用 torch.compile 避免 DataParallel 冲突）
# # ======================
# if __name__ == '__main__':
#     # 设备设置
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"✅ Device: {device}")
#     print(f"✅ PyTorch version: {torch.__version__}")
    
#     if torch.cuda.is_available():
#         print(f"  → GPU count: {torch.cuda.device_count()}")
#         for i in range(torch.cuda.device_count()):
#             print(f"  → GPU {i}: {torch.cuda.get_device_name(i)}")
    
#     # 路径配置
#     ROOT_DIR = "/kaggle/input/emotion-classfication/fer_data/fer_data"
#     if not os.path.exists(ROOT_DIR):
#         print(f"❌ Path not found: {ROOT_DIR}")
#         # 如果路径不存在，尝试其他可能路径
#         candidates = [
#             "/kaggle/input/fer-data/fer_data",
#             "/kaggle/input/fer2013/fer_data",
#             "./fer_data"
#         ]
#         for cand in candidates:
#             if os.path.exists(cand):
#                 ROOT_DIR = cand
#                 print(f"✅ Found data at: {ROOT_DIR}")
#                 break
    
#     train_path = Path(ROOT_DIR) / "train"
#     test_path = Path(ROOT_DIR) / "test"
    
#     # 检查路径存在性
#     if not train_path.exists():
#         print(f"❌ Train path not found: {train_path}")
#         print("Available directories:")
#         for item in Path(ROOT_DIR).iterdir():
#             print(f"  - {item}")
#         raise FileNotFoundError(f"Train directory not found at {train_path}")
    
#     # 创建变换
#     train_transform, val_transform = create_transforms()
    
#     # 创建数据集
#     try:
#         full_train_dataset = FERDataset(train_path, "train", transform=None)
#         print(f"✅ Loaded {len(full_train_dataset)} training samples")
#     except Exception as e:
#         print(f"❌ Error loading dataset: {e}")
#         raise
    
#     train_dataset = FERDataset(train_path, "train", transform=train_transform)
#     val_dataset = FERDataset(train_path, "train", transform=val_transform)
#     test_dataset = FERDataset(test_path, "test", transform=val_transform)
    
#     # 划分训练/验证集
#     total = len(full_train_dataset)
#     val_ratio = 0.2
#     val_size = int(total * val_ratio)
#     train_size = total - val_size
#     generator = torch.Generator().manual_seed(42)
#     indices = torch.randperm(total, generator=generator).tolist()
#     train_indices = indices[:train_size]
#     val_indices = indices[train_size:]
    
#     train_subset = Subset(train_dataset, train_indices)
#     val_subset = Subset(val_dataset, val_indices)
    
#     # T4*2 专属配置
#     if torch.cuda.device_count() > 1:
#         BATCH_SIZE = 256  # 双卡
#         print(f"🚀 Using 2x T4 GPUs with batch_size={BATCH_SIZE}")
#     else:
#         BATCH_SIZE = 128  # 单卡
#         print(f"🚀 Using single GPU with batch_size={BATCH_SIZE}")
    
#     NUM_WORKERS = min(4, os.cpu_count() or 4)  # 安全值
    
#     train_loader = DataLoader(
#         train_subset, 
#         batch_size=BATCH_SIZE, 
#         shuffle=True,
#         num_workers=NUM_WORKERS,
#         pin_memory=True,
#         persistent_workers=True,
#         drop_last=True
#     )
#     val_loader = DataLoader(
#         val_subset, 
#         batch_size=BATCH_SIZE, 
#         shuffle=False,
#         num_workers=NUM_WORKERS,
#         pin_memory=True,
#         persistent_workers=True
#     )
#     test_loader = DataLoader(
#         test_dataset, 
#         batch_size=BATCH_SIZE, 
#         shuffle=False,
#         num_workers=NUM_WORKERS,
#         pin_memory=True
#     )
    
#     print(f"📊 Dataset sizes: Train={len(train_subset)}, Val={len(val_subset)}, Test={len(test_dataset)}")
    
#     # 模型初始化
#     model = AdvancedCNN(pretrained=True).to(device)
    
#     # T4*2 多卡加速 (使用 DataParallel，禁用 torch.compile)
#     if torch.cuda.device_count() > 1:
#         model = nn.DataParallel(model)
#         print(f"ParallelGroup: Using {torch.cuda.device_count()} GPUs")
#         # ✅ 关键：不使用 torch.compile，避免 DataParallel 冲突
#         print("⚠️ torch.compile disabled for DataParallel compatibility")
#     else:
#         # 单卡时可启用 compile
#         if hasattr(torch, 'compile') and torch.__version__ >= '2.0':
#             try:
#                 model = torch.compile(model, mode="reduce-overhead")
#                 print("✅ torch.compile enabled (single GPU)")
#             except Exception as e:
#                 print(f"⚠️ torch.compile failed: {e}")
    
#     # 优化器 & 损失
#     criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
#     optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4, betas=(0.9, 0.999))
    
#     # 训练
#     print("\n🚀 Starting high-performance training...")
#     best_model_path, model = train_model(
#         model, train_loader, val_loader, optimizer, criterion,
#         epochs=25, device=device
#     )
    
#     # 测试（确保使用单卡模型）
#     if hasattr(model, "module"):
#         # 提取单卡模型用于测试
#         test_model = AdvancedCNN().to(device)
#         test_model.load_state_dict(model.module.state_dict())
#         model = test_model
    
#     # 测试
#     print("\n🧪 Running inference on test set...")
   

#     test(model, test_loader, device, output_csv='submission.csv')

✅ Device: cuda
✅ PyTorch version: 2.6.0+cu124
  → GPU count: 2
  → GPU 0: Tesla T4
  → GPU 1: Tesla T4
✅ Loaded 28275 training samples
🚀 Using 2x T4 GPUs with batch_size=256
📊 Dataset sizes: Train=22620, Val=5655, Test=7065
ParallelGroup: Using 2 GPUs
⚠️ torch.compile disabled for DataParallel compatibility

🚀 Starting high-performance training...
Epoch 01/25 | Time: 52.8s | LR: 2.99e-04 | Train Loss: 1.4619 | Acc: 45.53% | Val Loss: 1.2758 | Acc: 55.63%
  ✅ New best model saved @ 55.63%
Epoch 02/25 | Time: 50.5s | LR: 2.95e-04 | Train Loss: 1.2417 | Acc: 59.02% | Val Loss: 1.2111 | Acc: 60.21%
  ✅ New best model saved @ 60.21%
Epoch 03/25 | Time: 50.6s | LR: 2.89e-04 | Train Loss: 1.1708 | Acc: 62.73% | Val Loss: 1.1359 | Acc: 63.48%
  ✅ New best model saved @ 63.48%
Epoch 04/25 | Time: 50.5s | LR: 2.81e-04 | Train Loss: 1.1251 | Acc: 65.15% | Val Loss: 1.1368 | Acc: 63.87%
  ✅ New best model saved @ 63.87%
Epoch 05/25 | Time: 50.6s | LR: 2.71e-04 | Train Loss: 1.0856 | Acc: 67.44% | 

In [ ]:
if __name__ == '__main__':
    # ======================
    # 设备
    # ======================
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"✅ Device: {device}")
    print(f"✅ PyTorch: {torch.__version__}")

    if torch.cuda.is_available():
        print(f"  → GPU count: {torch.cuda.device_count()}")
        for i in range(torch.cuda.device_count()):
            print(f"  → GPU {i}: {torch.cuda.get_device_name(i)}")

    # ======================
    # 数据路径
    # ======================
    ROOT_DIR = "/kaggle/input/emotion-classfication/fer_data/fer_data"
    train_path = Path(ROOT_DIR) / "train"
    test_path  = Path(ROOT_DIR) / "test"

    if not train_path.exists():
        raise FileNotFoundError(f"Train path not found: {train_path}")

    # ======================
    # 数据增强
    # ======================
    train_transform, val_transform = create_transforms()

    # ======================
    # 数据集
    # ======================
    full_train_dataset = FERDataset(train_path, "train", transform=None)
    train_dataset = FERDataset(train_path, "train", transform=train_transform)
    val_dataset   = FERDataset(train_path, "train", transform=val_transform)
    test_dataset  = FERDataset(test_path,  "test",  transform=val_transform)

    print(f"✅ Total train samples: {len(full_train_dataset)}")

    # ======================
    # 划分 Train / Val
    # ======================
    total = len(full_train_dataset)
    val_ratio = 0.2
    val_size = int(total * val_ratio)
    train_size = total - val_size

    generator = torch.Generator().manual_seed(42)
    indices = torch.randperm(total, generator=generator).tolist()
    train_indices = indices[:train_size]
    val_indices   = indices[train_size:]

    train_subset = Subset(train_dataset, train_indices)
    val_subset   = Subset(val_dataset, val_indices)

    # ======================
    # DataLoader
    # ======================
    if torch.cuda.device_count() > 1:
        BATCH_SIZE = 256
        print("🚀 Using multi-GPU batch_size=256")
    else:
        BATCH_SIZE = 128
        print("🚀 Using single-GPU batch_size=128")

    NUM_WORKERS = min(4, os.cpu_count() or 4)

    train_loader = DataLoader(
        train_subset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        persistent_workers=True,
        drop_last=True
    )

    val_loader = DataLoader(
        val_subset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        persistent_workers=True
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True
    )

    print(f"📊 Train={len(train_subset)} | Val={len(val_subset)} | Test={len(test_dataset)}")

    # ======================
    # 模型
    # ======================
    model = AdvancedCNN(pretrained=True).to(device)

    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        print("⚠️ DataParallel enabled (torch.compile disabled)")
    else:
        if hasattr(torch, "compile") and torch.__version__ >= "2.0":
            try:
                model = torch.compile(model, mode="reduce-overhead")
                print("✅ torch.compile enabled")
            except Exception as e:
                print(f"⚠️ torch.compile failed: {e}")

    # ======================
    # ⭐ 关键修改 ①：类别权重
    # ======================
    from collections import Counter

    labels = [label for _, label in full_train_dataset.samples]
    counter = Counter(labels)
    print("📊 Class distribution:", counter)

    class_weights = torch.tensor(
        [1.0 / counter[i] for i in range(6)],
        dtype=torch.float
    )
    class_weights = class_weights / class_weights.sum() * 6
    class_weights = class_weights.to(device)

    # ======================
    # ⭐ 关键修改 ②：Focal Loss（带权重）
    # ======================
    criterion = FocalLoss(
        gamma=2.0,
        alpha=class_weights
    )

    # （如果你想用 Weighted CE，改成下面这行即可）
    # criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.05)

    # ======================
    # 优化器
    # ======================
    optimizer = optim.AdamW(
        model.parameters(),
        lr=3e-4,
        weight_decay=1e-4,
        betas=(0.9, 0.999)
    )

    # ======================
    # 训练（内部已启用 MixUp）
    # ======================
    print("\n🚀 Start Training (Focal + MixUp)...")
    model = train_model(
        model,
        train_loader,
        val_loader,
        optimizer,
        criterion,
        epochs=25,
        device=device
    )

    # ======================
    # 多卡 → 单卡（测试用）
    # ======================
    if hasattr(model, "module"):
        single_model = AdvancedCNN(pretrained=False).to(device)
        single_model.load_state_dict(model.module.state_dict())
        model = single_model

    # ======================
    # ⭐ 关键修改 ③：TTA 测试
    # ======================
    print("\n🧪 Inference with TTA...")
    test(
        model,
        test_loader,
        device,
        output_csv="submission.csv"
    )

    print("\n🎉 Done! Focal + MixUp + TTA pipeline finished.")
